## Employee Attrition Prediction using Naive Bayes 

### Problem Statement:
We have historical Employee Data with a number of features about each employee. The ask from the organisation is to predict whether or not an Employee, given it's attributes values, will attrite or not. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

### Import Employee Data into a DataFrame

In [2]:
df = pd.read_csv("HR-Employee-Attrition.csv")

In [3]:
df

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,No,Travel_Frequently,884,Research & Development,23,2,Medical,1,2061,...,3,80,1,17,3,3,5,2,0,3
1466,39,No,Travel_Rarely,613,Research & Development,6,1,Medical,1,2062,...,1,80,1,9,5,3,7,7,1,7
1467,27,No,Travel_Rarely,155,Research & Development,4,3,Life Sciences,1,2064,...,2,80,1,6,0,3,6,2,0,3
1468,49,No,Travel_Frequently,1023,Sales,2,3,Medical,1,2065,...,4,80,0,17,3,2,9,6,0,8


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1470 non-null   int64 
 1   Attrition                 1470 non-null   object
 2   BusinessTravel            1470 non-null   object
 3   DailyRate                 1470 non-null   int64 
 4   Department                1470 non-null   object
 5   DistanceFromHome          1470 non-null   int64 
 6   Education                 1470 non-null   int64 
 7   EducationField            1470 non-null   object
 8   EmployeeCount             1470 non-null   int64 
 9   EmployeeNumber            1470 non-null   int64 
 10  EnvironmentSatisfaction   1470 non-null   int64 
 11  Gender                    1470 non-null   object
 12  HourlyRate                1470 non-null   int64 
 13  JobInvolvement            1470 non-null   int64 
 14  JobLevel                

#### Observation: No Missing Values in the Data

#### Replace 'Yes' and 'No' in target feature by 1 and 0

In [5]:
df.Attrition.replace({"Yes":1,"No":0}, inplace=True)

#### Delete the following Features as their values are same across all observations

In [6]:
df.drop(columns=['EmployeeCount','StandardHours'], inplace=True)
df.columns

Index(['Age', 'Attrition', 'BusinessTravel', 'DailyRate', 'Department',
       'DistanceFromHome', 'Education', 'EducationField', 'EmployeeNumber',
       'EnvironmentSatisfaction', 'Gender', 'HourlyRate', 'JobInvolvement',
       'JobLevel', 'JobRole', 'JobSatisfaction', 'MaritalStatus',
       'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked', 'Over18',
       'OverTime', 'PercentSalaryHike', 'PerformanceRating',
       'RelationshipSatisfaction', 'StockOptionLevel', 'TotalWorkingYears',
       'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsAtCompany',
       'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager'],
      dtype='object')

### Create lists of Categorical Features and Numerical Features

In [7]:
cat_col = df.select_dtypes(exclude=np.number).columns
num_col = df.select_dtypes(include=np.number).columns

In [8]:
for i in cat_col:
    print(f"\n=================> {i} \n")
    print(df[i].value_counts())


=================> BusinessTravel 

Travel_Rarely        1043
Travel_Frequently     277
Non-Travel            150
Name: BusinessTravel, dtype: int64

=================> Department 

Research & Development    961
Sales                     446
Human Resources            63
Name: Department, dtype: int64

=================> EducationField 

Life Sciences       606
Medical             464
Marketing           159
Technical Degree    132
Other                82
Human Resources      27
Name: EducationField, dtype: int64

=================> Gender 

Male      882
Female    588
Name: Gender, dtype: int64

=================> JobRole 

Sales Executive              326
Research Scientist           292
Laboratory Technician        259
Manufacturing Director       145
Healthcare Representative    131
Manager                      102
Sales Representative          83
Research Director             80
Human Resources               52
Name: JobRole, dtype: int64

=================> MaritalStatus 

Marri

### Create Dummy Variables for Categorical Variable

In [9]:
encoded_cat_col = pd.get_dummies(df[cat_col], drop_first=True)

In [10]:
final_model = pd.concat([df[num_col],encoded_cat_col], axis = 1)

In [11]:
final_model

,Age,Attrition,DailyRate,DistanceFromHome,Education,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,...,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Married,MaritalStatus_Single,OverTime_Yes
0,41,1,1102,1,2,1,2,94,3,2,...,0,0,0,0,0,1,0,0,1,1
1,49,0,279,8,1,2,3,61,2,2,...,0,0,0,0,1,0,0,1,0,0
2,37,1,1373,2,2,4,4,92,2,1,...,1,0,0,0,0,0,0,0,1,1
3,33,0,1392,3,4,5,4,56,3,1,...,0,0,0,0,1,0,0,1,0,1
4,27,0,591,2,1,7,1,40,3,1,...,1,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,0,884,23,2,2061,3,41,4,2,...,1,0,0,0,0,0,0,1,0,0
1466,39,0,613,6,1,2062,4,42,2,3,...,0,0,0,0,0,0,0,1,0,0
1467,27,0,155,4,3,2064,2,87,4,2,...,0,0,1,0,0,0,0,1,0,1
1468,49,0,1023,2,3,2065,4,63,2,2,...,0,0,0,0,0,1,0,1,0,0


In [12]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report

### Segrgate X and y Features, Create Train and Test Sets

In [13]:
x = final_model.drop(columns="Attrition")
y = final_model["Attrition"]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=10)

In [14]:
from sklearn.naive_bayes import GaussianNB

### Create our Gaussian Naive Bayes Model

In [15]:
model = GaussianNB()

#### Train and Predict using Training Data

In [16]:
model.fit(x_train, y_train)
train_Pred = model.predict(x_train)

In [17]:
metrics.confusion_matrix(y_train,train_Pred)

array([[749, 127],
       [ 58,  95]])

In [23]:
Accuracy_percent_train = (metrics.accuracy_score(y_train,train_Pred)) * 100
Accuracy_percent_train

82.02137998056365

#### Our model is able to predict with 82% accuracy from Training Data

### Predict using Test Data

In [19]:
test_Pred = model.predict(x_test)

In [20]:
metrics.confusion_matrix(y_test,test_Pred)

array([[300,  57],
       [ 28,  56]])

In [21]:
Accuracy_percent_test = (metrics.accuracy_score(y_test,test_Pred))*100
Accuracy_percent_test

80.72562358276643

#### Our model is able to predict with 80.7% accuracy from Test Data

In [28]:
list(zip(y_test, test_Pred))[0:20]

[(0, 0),
 (1, 1),
 (0, 0),
 (0, 0),
 (0, 0),
 (1, 1),
 (0, 0),
 (1, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 1),
 (0, 0),
 (1, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0)]

In [22]:
print(classification_report(y_test, test_Pred))

              precision    recall  f1-score   support

           0       0.91      0.84      0.88       357
           1       0.50      0.67      0.57        84

    accuracy                           0.81       441
   macro avg       0.71      0.75      0.72       441
weighted avg       0.83      0.81      0.82       441

